In [108]:
import geopandas as gpd
import pandas as pd
import folium as fl

In [109]:
results = pd.read_csv('resultats.csv')

In [110]:
geojson = gpd.read_file('./contours-france-entiere-latest.geojson')

In [111]:
geojson = geojson.rename(columns={'id_bv': 'id_bureau'})
merged = pd.merge(results, geojson, on='id_bureau')

In [ ]:
from folium.plugins import MarkerCluster
import threading

m = fl.Map(location=[46.603354, 1.888334], zoom_start=5, prefer_canvas=True)
markerCluster = MarkerCluster().add_to(m)

def addMarker(row):
    macron = '% Voix/Exp'
    lepen = '% Voix/Exp.1'
    centroid = row['geometry'].centroid
    prompt = 'Macron : ' + str(row.loc[macron]) + '%<br>Lepen: ' + str(row.loc[lepen]) + '%'
    iframe = fl.IFrame(prompt, width=200, height=80)
    popup = fl.Popup(iframe)
    fl.Marker(
        location=[centroid.y, centroid.x],
        tooltip=row['id_bureau'],
        popup=popup,
        icon=fl.Icon("purple" if row[macron] > row[lepen] else "blue")
    ).add_to(markerCluster)

def iterativeAddMarker():
    for _, row in merged.iterrows():
        addMarker(row)

iterativeAddMarker()

In [ ]:
m.save('./index.html')